### Import Modules

In [138]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from typing import Optional
import csv

## Preprocessing

In [139]:
# Read data from the csv file
df_raw = pd.read_csv('./data/survey_results_public.csv', sep=',')
df_raw

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Difficult,NaN
2,3,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Friend or family member...,Technical documentation;Blogs;Programming Game...,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,40205.0
3,4,I am a developer by profession,"Employed, full-time",Fully remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., Universit...",NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,215232.0
4,5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;O...,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73263,73264,I am a developer by profession,"Employed, full-time",Fully remote,Freelance/contract work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Written Tutorial...,Udemy,8,...,30-60 minutes a day,Less than 15 minutes a day,Just right,DevOps function;Microservices;Developer portal...,Yes,Yes,Yes,Too long,Easy,NaN
73264,73265,I am a developer by profession,"Employed, full-time",Full in-person,Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Written Tutorial...,Coursera;Udemy;Udacity,6,...,15-30 minutes a day,60-120 minutes a day,Very long,None of these,No,Yes,Yes,Too long,Easy,NaN
73265,73266,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",Hobby;School or academic work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Programming Games;Stac...,Udemy;Codecademy;Pluralsight;edX,42,...,30-60 minutes a day,60-120 minutes a day,Just right,None of these,No,No,No,Appropriate in length,Easy,NaN
73266,73267,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;On the job training,NaN,NaN,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN


In [140]:
# select only those columns that are important for this analysis
df = df_raw.copy()
df = df[['ConvertedCompYearly', 'EdLevel', 'YearsCode', 'YearsCodePro', 'OrgSize', 'Age', 'MainBranch', 'RemoteWork', 'DevType', 'Country', 'LanguageHaveWorkedWith', 'Gender', 'Ethnicity']]

# rename columns
df = df.rename(columns={
    'ConvertedCompYearly':'Salary',
    'MainBranch':'Employment',
    'LanguageHaveWorkedWith':'ProgLanguage'
})

# Replace unnecessarily long entries with shorter ones
df['Employment'] = df['Employment'].replace(
    to_replace=[
        "I am a developer by profession",
        "I am learning to code",
        "I am not primarily a developer, but I write code sometimes as part of my work",
        "I code primarily as a hobby",
        "I used to be a developer by profession, but no longer am",
        "None of these"
    ],
    value=[
        "profession",
        "student",
        "part of work",
        "hobby",
        "ex profession",
        "unknown"
    ]
)

df['EdLevel'] = df['EdLevel'].replace(
    to_replace=[
        "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",
        "Bachelor’s degree (B.A., B.S., B.Eng., etc.)",
        "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",
        "Other doctoral degree (Ph.D., Ed.D., etc.)",
        "Some college/university study without earning a degree",
        "Something else",
        "Professional degree (JD, MD, etc.)",
        "Primary/elementary school",
        "Associate degree (A.A., A.S., etc.)"
    ],
    value=[
        "Secondary school",
        "Bachelor’s degree",
        "Master’s degree",
        "Doctoral degree",
        "University courses",
        "Unknown",
        "Professional degree",
        "Primary school",
        "Associate degree"
    ]
)

df['YearsCode'] = df['YearsCode'].replace(
    to_replace=[
        "More than 50 years",
        "Less than 1 year"
    ],
    value=[
        "50",
        "0",
    ]
)

df['YearsCodePro'] = df['YearsCodePro'].replace(
    to_replace=[
        "More than 50 years",
        "Less than 1 year"
    ],
    value=[
        "50",
        "0",
    ]
)

df['OrgSize'] = df['OrgSize'].replace(
    to_replace=[
        "Just me - I am a freelancer, sole proprietor, etc.",
        "2 to 9 employees",
        "10 to 19 employees",
        "20 to 99 employees",
        "100 to 499 employees",
        "500 to 999 employees",
        "1,000 to 4,999 employees",
        "5,000 to 9,999 employees",
        "10,000 or more employees",
        "I don’t know"
    ],
    value=[
        "1",
        "2 to 9",
        "10 to 19",
        "20 to 99",
        "100 to 499","500 to 999",
        "1,000 to 4,999",
        "5,000 to 9,999",
        "10,000 or more",
        "unknown"
    ]
)

df['RemoteWork'] = df['RemoteWork'].replace(
    to_replace=[
        "Full in-person",
        "Hybrid (some remote, some in-person)",
        "Fully remote"
    ],
    value=[
        "in-person",
        "hybrid",
        "remote"
    ]
)

df['Age'] = df['Age'].replace(
    to_replace=[
        "Under 18 years old",
        "18-24 years old",
        "25-34 years old",
        "35-44 years old",
        "45-54 years old",
        "55-64 years old",
        "65 years or older",
        "Prefer not to say"
    ],
    value=[
        "under 18",
        "18-24",
        "25-34",
        "35-44",
        "45-54",
        "55-64",
        "over 65",
        "unknown"
    ]
)

# Convert the gender values into 'Man', 'Woman', 'Prefer not to say' and 'Other'
df.loc[~df['Gender'].isin(['Man', 'Woman', 'Prefer not to say']), 'Gender'] = "Other"

# Convert columns into numeric values
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')
df['YearsCode'] = pd.to_numeric(df['YearsCode'], errors='coerce')
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# delete all rows with NaN in Salary as those entries can't be used
df = df[df['Salary'].isnull() == False]

df_ger = df.copy()
df_ger = df_ger[df_ger['Country'] == "Germany"]

In [141]:
df_ger.loc[(df_ger['Salary'] > 1000000) & (df_ger['Country'] == "Germany")]

,Salary,EdLevel,YearsCode,YearsCodePro,OrgSize,Age,Employment,RemoteWork,DevType,Country,ProgLanguage,Gender,Ethnicity
1705,1190148.0,Master’s degree,17.0,11.0,unknown,35-44,profession,hybrid,"Developer, game or graphics",Germany,C++;Python,Man,I don't know
2091,1023780.0,Master’s degree,16.0,8.0,100 to 499,25-34,profession,remote,"Engineer, site reliability;DevOps specialist;C...",Germany,Bash/Shell;C#;Groovy;Java;Rust;TypeScript,Man,European
5508,1151748.0,Master’s degree,20.0,7.0,"10,000 or more",25-34,profession,remote,Data scientist or machine learning specialist;...,Germany,Bash/Shell;Python;SQL,Man,White;European
5524,1394904.0,Master’s degree,35.0,24.0,2 to 9,45-54,profession,hybrid,"Developer, full-stack",Germany,C++;Delphi;HTML/CSS;Java;JavaScript;PHP;SQL;VBA,Man,European
6671,1023780.0,University courses,25.0,25.0,1,45-54,profession,remote,"Developer, front-end;Developer, full-stack;Dev...",Germany,Bash/Shell;HTML/CSS;Java;JavaScript;Kotlin;PHP...,Man,White;European
8542,1105680.0,Master’s degree,12.0,10.0,"10,000 or more",25-34,profession,in-person,Data scientist or machine learning specialist;...,Germany,Bash/Shell;C++;Java;JavaScript;MATLAB;Python;VBA,Man,Indian
9341,1023780.0,University courses,12.0,8.0,10 to 19,25-34,profession,hybrid,"Developer, full-stack",Germany,Bash/Shell;HTML/CSS;JavaScript;SQL;TypeScript,Man,European;African;Multiracial;Biracial
17123,1279728.0,Bachelor’s degree,14.0,10.0,20 to 99,25-34,profession,remote,"Developer, front-end;Developer, full-stack;Dev...",Germany,Bash/Shell;C#;F#;HTML/CSS;JavaScript;Lua;Power...,Man,White;European;Asian;Biracial
18119,1791612.0,Secondary school,22.0,17.0,"10,000 or more",35-44,profession,hybrid,"Developer, front-end;Engineer, data;Developer,...",Germany,C;C#;C++;Go;HTML/CSS;Java;JavaScript;Kotlin;Ob...,Man,White;European
18472,1215744.0,Bachelor’s degree,17.0,12.0,500 to 999,25-34,profession,remote,"Developer, mobile",Germany,Bash/Shell;Swift,Other,NaN


#### Cut outliers

In [147]:
upper_05 = int(np.quantile(df_ger['Salary'], q=0.95))
upper_10 = int(np.quantile(df_ger['Salary'], q=0.9))
lower_05 = int(np.quantile(df_ger['Salary'], q=0.05))
lower_10 = int(np.quantile(df_ger['Salary'], q=0.1))

""" print(f"0.95 quantil: {upper_05 : >6}")
print(f"0.90 quantil: {upper_10 : >6}")
print(f"0.05 quantil: {lower_05 : >6}") """

#_, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(20, 6))

# These should rather be in the plots file
""" 
ax1.violinplot(df_ger['Salary'])
ax1.boxplot(df_ger['Salary'], showmeans=True, meanline=True)
ax1.set_ylabel("Salary")
ax1.set(title='all data')

ax2.violinplot(df_ger[df_ger['Salary'].between(lower_05, upper_05)]['Salary'])
ax2.boxplot(df_ger[df_ger['Salary'].between(lower_05, upper_05)]['Salary'], showmeans=True, meanline=True)
ax2.set_ylabel("Salary")
ax2.set(title='lower: 05 | upper: 05')

ax3.violinplot(df_ger[df_ger['Salary'].between(lower_05, upper_10)]['Salary'])
ax3.boxplot(df_ger[df_ger['Salary'].between(lower_05, upper_10)]['Salary'], showmeans=True, meanline=True)
ax3.set_ylabel("Salary")
ax3.set(title='lower: 05 | upper: 10')

ax4.violinplot(df_ger[df_ger['Salary'].between(lower_10, upper_10)]['Salary'])
ax4.boxplot(df_ger[df_ger['Salary'].between(lower_10, upper_10)]['Salary'], showmeans=True, meanline=True)
ax4.set_ylabel("Salary")
ax4.set(title='lower: 05 | upper: 10')

plt.show() """

df_ger = df_ger[df_ger['Salary'].between(lower_10, upper_10)]

In [143]:
df_ger = df_ger.copy()

# delete all rows where coding is not their profession
df_ger = df_ger[df_ger['Employment'] == 'profession']

# select important features
df_ger = df_ger[['Salary', 'EdLevel', 'YearsCode', 'OrgSize', 'Age', 'RemoteWork', 'ProgLanguage', 'DevType']]

df_ger['Age'] = df_ger['Age'].replace(
    to_replace=[
        "unknown",
        "under 18",
        "18-24",
        "25-34",
        "35-44",
        "45-54",
        "55-64",
        "over 65",
    ],
    value=[
        None,
        1,
        2,
        3,
        4,
        5,
        6,
        7
    ]
)
df_ger['OrgSize'] = df_ger['OrgSize'].replace(
    to_replace=[
        "unknown",
        "1",
        "2 to 9",
        "10 to 19",
        "20 to 99",
        "100 to 499",
        "500 to 999",
        "1,000 to 4,999",
        "5,000 to 9,999",
        "10,000 or more"
    ],
    value=[
        None,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9
    ]
)

In [144]:
#### TOO MANY INDIVIDUAL PROGRAMMING LANGUAGES TO LOOK AT ALL!!

# df_prog = df_usa.copy()
# df_prog = df_prog[df_usa['Salary']<300000]
# df_prog['ProgLanguage'] = df_prog['ProgLanguage'].str.split(';')
# df_prog = df_prog.explode('ProgLanguage')
#
# df_prog['ProgLanguage'].unique()

### Encoding of the features

#### One-Hot Encoding

In [145]:
# one hot encoding 'EdLevel' and 'RemoteWork'; drop first to avoid the dummy variable trap
df_ger = pd.concat([df_ger, pd.get_dummies(df_ger['EdLevel'], drop_first=True)], axis=1)
df_ger = df_ger.drop(['EdLevel', 'Unknown'], axis=1)

df_ger = pd.concat([df_ger, pd.get_dummies(df_ger['RemoteWork'], drop_first=True)], axis=1)
df_ger = df_ger.drop(['RemoteWork'], axis=1)

# one hot encoding 'ProgLanguage' and 'DevType' where with ";" seperated list values
df_ger = pd.concat([df_ger, df_ger['ProgLanguage'].str.get_dummies(sep=';')], axis=1)
df_ger = df_ger.drop(['ProgLanguage'], axis=1)

#df_ger = pd.concat([df_ger, df_ger['DevType'].str.get_dummies(sep=';')], axis=1)
df_ger = df_ger.drop(['DevType'], axis=1)

df_ger.apply(pd.to_numeric, errors='ignore')

df_ger = df_ger.dropna()

#### Write preprocessed data to new file

In [146]:
csv_filename = 'preprocessed_survey_results.csv'

df_ger.to_csv('data/preprocessed_survey_results.csv')

print("-------------------------------")
print("Preprocessing done")
print("Number of samples in file " + csv_filename + ": " + str(len(df_ger)))
print("Attributes in file " + csv_filename + ": " + str(df_ger.columns.tolist()))
print("-------------------------------")


-------------------------------
Preprocessing done
Number of samples in file preprocessed_survey_results.csv: 2086
Attributes in file preprocessed_survey_results.csv: ['Salary', 'YearsCode', 'OrgSize', 'Age', 'Bachelor’s degree', 'Doctoral degree', 'Master’s degree', 'Primary school', 'Professional degree', 'Secondary school', 'University courses', 'in-person', 'remote', 'APL', 'Assembly', 'Bash/Shell', 'C', 'C#', 'C++', 'COBOL', 'Clojure', 'Crystal', 'Dart', 'Delphi', 'Elixir', 'Erlang', 'F#', 'Fortran', 'Go', 'Groovy', 'HTML/CSS', 'Haskell', 'Java', 'JavaScript', 'Julia', 'Kotlin', 'LISP', 'Lua', 'MATLAB', 'OCaml', 'Objective-C', 'PHP', 'Perl', 'PowerShell', 'Python', 'R', 'Ruby', 'Rust', 'SAS', 'SQL', 'Scala', 'Solidity', 'Swift', 'TypeScript', 'VBA']
-------------------------------
